In [18]:
import pandas as pd

df = pd.read_parquet('/Users/Macington/Documents/Projects/Project Gambit/Games/Storage/total_ply=57/data.parquet')

df

,game_id,pgn,ply,board_sum,centipawn_evaluation,board_sum_in_context
0,4.893091e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",NaN,18446462598732906495,NaN,1.844646e+19
1,4.893091e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",1.0,18446462599001337855,53.0,3.689293e+19
2,4.893091e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",2.0,18444210833547390975,75.0,5.533714e+19
3,4.893091e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",3.0,18444210833278955519,61.0,7.378135e+19
4,4.893091e+20,"[Event ""Mar del Plata""]\n[Site ""Mar del Plata""...",4.0,17867750080975532031,71.0,9.164910e+19
...,...,...,...,...,...,...
44398,4.788434e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",52.0,5956221751573777230,NaN,4.550185e+20
44399,4.788434e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",53.0,5956221751573777254,NaN,4.609748e+20
44400,4.788434e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",54.0,5956221614135872358,NaN,4.669310e+20
44401,4.788434e+20,"[Event ""Las Palmas""]\n[Site ""Las Palmas""]\n[Da...",55.0,5956221648361392998,NaN,4.728872e+20


In [22]:
import pyarrow.parquet as pq

# Directory containing the Parquet files
parquet_dir = "/Users/Macington/Documents/Projects/Project Gambit/Games/Storage"

# Read the schema of the first Parquet file in the directory
dataset = pq.ParquetDataset(parquet_dir)
schema = dataset.schema

# Print the schema
print(schema)


game_id: double
pgn: string
ply: double
board_sum: uint64
centipawn_evaluation: double
board_sum_in_context: double
total_ply: dictionary<values=int32, indices=int32, ordered=0>
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 986


In [ ]:
import pandas as pd
from pathlib import Path
import gc

def collect_board_sum_in_context_values(storage_directory, start_ply, end_ply):
    board_sum_in_context_values = {}
    for total_ply in range(1, 350 + 1):
        partition_path = Path(storage_directory, f"total_ply={total_ply}", "data.parquet")
        if not partition_path.exists():
            continue
        
        df = pd.read_parquet(partition_path)
        for board_sum_in_context, centipawn_evaluation in zip(df['board_sum_in_context'], df['centipawn_evaluation']):
            if pd.notnull(centipawn_evaluation) and centipawn_evaluation != 0:
                if board_sum_in_context in board_sum_in_context_values:
                    board_sum_in_context_values[board_sum_in_context].append(centipawn_evaluation)
                else:
                    board_sum_in_context_values[board_sum_in_context] = [centipawn_evaluation]

    # Compute the average for non-zero values
    for key, values in board_sum_in_context_values.items():
        board_sum_in_context_values[key] = sum(values) / len(values)

    return board_sum_in_context_values

def join_values(storage_directory, start_ply, end_ply, board_sum_in_context_values):
    for total_ply in range(start_ply, end_ply + 1):
        partition_path = Path(storage_directory, f"total_ply={total_ply}", "data.parquet")
        if not partition_path.exists():
            continue
        
        df = pd.read_parquet(partition_path)
        mask = df['centipawn_evaluation'].isnull()
        df.loc[mask, 'centipawn_evaluation'] = df.loc[mask, 'board_sum_in_context'].map(board_sum_in_context_values)
        df.to_parquet(partition_path)
        
        print(f"Updated partition for total_ply={total_ply}...")
        gc.collect()

def main():
    storage_directory = "/Users/Macington/Documents/Projects/Project Gambit/Games/Storage"
    start_ply = 47
    end_ply = 228

    # Collect board_sum_in_context values
    print("Collecting board_sum_in_context values...")
    board_sum_in_context_values = collect_board_sum_in_context_values(storage_directory, start_ply, end_ply)

    # Join the matching centipawn_evaluation values
    print("Joining matching centipawn_evaluation values...")
    join_values(storage_directory, start_ply, end_ply, board_sum_in_context_values)

    print("Processing completed!")

if __name__ == "__main__":
    main()
